## Importar Librerias

In [2]:
# Importación de las librerías necesarias
import numpy as np
import pandas as pd
# Puede que nos sirvan también
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression, Perceptron, Ridge
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, classification_report, roc_curve, auc
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

np.random.seed(0)  # Para mayor determinismo

## 1.Carga de datos

Al final del practico anterior obtuvimos un dataset enriquesido con features georeferenciadas. Es el momento de utilizarlo.

Exporta el dataframe obtenido en el ultimo practico a un csv y cargalo como dataframe.

In [72]:
dataframe = pd.read_csv('./data/curated_data.csv', sep='\t')
dataframe

,COD_ID,CONJ,MUN,ARE_LOC,geometry,UNI_TR_S,DIC,FIC,CNAE,ENE_01,...,n_close_con,is_urb,NODE,ENDNODE,g_degree,g_avg_ndeg,g_pagerank,g_dispersion,eigenvector_centrality,dist_to_trs_dijkstra
0,4_1040341,12737,2805802,NU,POINT (627712.6169779997 8776098.670127511),1_SESAQ_1,7.050,8.0,0,147,...,1,False,45297,44304,2,2.0,0.000012,"{'45298': 0.0, '45296': 0.0}",5.680527e-06,110
1,4_1040306,12737,2805802,NU,POINT (626353.4179798126 8777498.606078334),1_SESAQ_1,5.470,7.0,0,275,...,1,False,45268,44304,1,2.0,0.000007,{'45269': 0.0},2.625699e-06,126
2,4_1040035,12737,2805802,NU,POINT (635724.1370160332 8773322.662057359),1_SESAQ_1,7.130,7.0,0,1732,...,14,False,46283,44304,2,2.0,0.000012,"{'46285': 0.0, '46282': 0.0}",4.072908e-07,240
3,4_1040036,12737,2805802,NU,POINT (638696.4859638879 8773991.73208276),1_SESAQ_1,5.190,6.0,0,692,...,9,False,46525,44304,1,3.0,0.000006,{'46408': 0.0},7.984421e-07,233
4,4_1040038,12737,2805802,NU,POINT (638310.7990282972 8773035.106081862),1_SESAQ_1,5.840,7.0,0,1297,...,10,False,46406,44304,2,2.0,0.000012,"{'46438': 0.0, '46517': 0.0}",3.746185e-07,261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8828,8_1080578,12730,2802809,NU,POINT (653013.3989592987 8739601.889035663),1_SECON_1,21.510,10.0,0,1206,...,12,False,65017,68429,2,2.0,0.000012,"{'65016': 0.0, '65018': 0.0}",4.457296e-07,321
8829,8_1080577,12730,2802809,NU,POINT (653309.3439843655 8739968.353031982),1_SECON_1,21.510,10.0,0,1469,...,8,False,64978,68429,2,2.0,0.000012,"{'65005': 0.0, '65004': 0.0}",2.614774e-07,334
8830,12_1120421,12730,2807600,NU,POINT (644710.9110109408 8734252.079058295),1_SECON_1,21.130,9.0,0,189,...,1,False,65134,68429,1,2.0,0.000007,{'65245': 0.0},1.424013e-07,434
8831,8_1080557,12730,2802809,NU,POINT (673768.0680233241 8730109.019107511),1_SECON_1,11.435,6.0,0,69,...,4,False,680043,71128,1,2.0,0.000007,{'680042': 0.0},1.931153e-07,139


Ahora debemos eliminar variables que no aporte informacion por ejemplo:


*   Indices y IDs
*   Referencias a otras tablas
*   Variables compuestas (listas, jsons, geometrias)

Identifica las variables que cumplen este criterio y eliminelas del dataset. Retira tambien la variable de salida que no van a utilizar FIC o DIC.



In [163]:
# Elimino índices, IDs y variables compuestas.
dataframe.drop(
    columns=[
        'COD_ID', 'PAC_1', 'PAC_2', 'geometry', 'TR_S_geometry',
        'g_dispersion'
    ],
    inplace=True,
    errors='ignore'
)

# Elimino las siguientes variables dado que:
# TR_S_BARR_3 == '' para todas las filas,
# DIST == 46 para todas las filas,
# CONF == 'RA' para todas las filas excepto una,
# POSTO == 'PT' para todas las filas,
# BANC == 0 para todas la filas,
# MRT == 0 para todas la filas,
# DESCR = '' para todas las filas,
# SIT_ATIV = 'AT' para todas las filas excepto una,
# TIP_UNID = 38 para todas las filas,
# ATRB_PER = 1 para casi todas las filas,
# CAP_CHA = 1 para casi todas las filas,
dataframe.drop(
    columns=[
        'TR_S_BARR_3', 'DIST', 'CONF', 'POSTO', 'BANC', 'MRT', 'DESCR',
        'SIT_ATIV', 'TIP_UNID', 'ATRB_PER', 'CAP_CHA'
    ],
    inplace=True,
    errors='ignore'
)

# Elimino DIC (vamos a usar FIC).
dataframe.drop(columns=['DIC'], inplace=True, errors='ignore')

# Con respecto a otras variables:
# DAT_CON y TR_S_DAT_CON son fechas. Nos sirven?
# Qué es NODE? No me acuerdo...
dataframe

,CONJ,MUN,ARE_LOC,UNI_TR_S,FIC,CNAE,ENE_01,ENE_02,ENE_03,ENE_04,...,dist_untrs,n_close_con,is_urb,NODE,ENDNODE,g_degree,g_avg_ndeg,g_pagerank,eigenvector_centrality,dist_to_trs_dijkstra
0,12737,2805802,NU,1_SESAQ_1,8.0,0,147,154,150,115,...,8.798533e+06,1,False,45297,44304,2,2.0,0.000012,5.680527e-06,110
1,12737,2805802,NU,1_SESAQ_1,7.0,0,275,236,263,256,...,8.799832e+06,1,False,45268,44304,1,2.0,0.000007,2.625699e-06,126
2,12737,2805802,NU,1_SESAQ_1,7.0,0,1732,1543,1413,1647,...,8.796339e+06,14,False,46283,44304,2,2.0,0.000012,4.072908e-07,240
3,12737,2805802,NU,1_SESAQ_1,6.0,0,692,650,604,697,...,8.797222e+06,9,False,46525,44304,1,3.0,0.000006,7.984421e-07,233
4,12737,2805802,NU,1_SESAQ_1,7.0,0,1297,1175,1160,1370,...,8.796240e+06,10,False,46406,44304,2,2.0,0.000012,3.746185e-07,261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8828,12730,2802809,NU,1_SECON_1,10.0,0,1206,1382,1062,1039,...,8.763978e+06,12,False,65017,68429,2,2.0,0.000012,4.457296e-07,321
8829,12730,2802809,NU,1_SECON_1,10.0,0,1469,1692,1402,1422,...,8.764366e+06,8,False,64978,68429,2,2.0,0.000012,2.614774e-07,334
8830,12730,2807600,NU,1_SECON_1,9.0,0,189,181,191,196,...,8.758028e+06,1,False,65134,68429,1,2.0,0.000007,1.424013e-07,434
8831,12730,2802809,NU,1_SECON_1,6.0,0,69,63,58,57,...,8.756085e+06,4,False,680043,71128,1,2.0,0.000007,1.931153e-07,139


## 2. Preprosecamiento y particion de datos

Siempre se recomienda partir los datos antes de realizar el preprocesamiento, y guardar el pipeline de transformacion para aplicar en operaciones futuras.

Sin embargo algunas operaciones principalmente la codificacion de variables puede realizarse antes de hacer la particion ya que no representa una interaccion entre las filas.

Genere una particion de datos en sets de entrenamiento y validacion.

KeyError: "['TR_S_BARR_3' 'DIST' 'CONF' 'POSTO' 'BANC' 'MRT' 'DESCR' 'SIT_ATIV'\n 'TIP_UNID' 'ATRB_PER' 'CAP_CHA'] not found in axis"

Un elemento que tenemos que siempre debemos tener en cuenta es que el procesamiento que debemos hacer sobre los datos esta relacionado con el algoritmo que vayas a implementar.

## 3. Preparacion de datos para regresion lineal

Los algoritmos de regresion solo aceptan entradas numericas, por lo tanto el primer paso que debemos realizar es codificar las variables categoricas numericamente.

Adicionalmente las diferencias en las dimensiones absolutas de las variables van a tener un impacto muy pronunciado en los coeficientes calculados por lo que tambien se recomienda realizar la normalizacion de las variables. 

Esto puede hacerse de varias maneras:

*   Media/Mediana y desviacion standard
*   Minimo y Maximo (Cuidado con los outliers)
*   De 1 a -1

Implemente una funcion o pipeline que aplique codificacion de variables categoricas y luego normalizacion. Recuerde que los parametros de normalizacion se calculan usando unicamente los valores de entrenamiento.

## 4. Regresion lineal simple y regularizada.

Con los datos preprocesados obtenidos en el punto anterior implemente una regresion lineal. 

### Evaluacion del modelo

Para evaluar el modelo calcule R cuadrado, error medio absoluto y error medio cuadrado. Tanto de entrenamiento como de validacion.

### Analisis de residuos

Otro punto importante en la evaluacion de los modelos de regresion, es no solo saber cuanto es el error sino como se distribuye. Para eso realizamos un analisis de residuos. Los residuos son simplemente las desviaciones de la prediccion al valor real.

Primero grafique la distribucion de los residuos.

Esta distribucion debe estar centrada en cero y deberia tender a ser simetrica.

**A continuacion genere un scatter plot entre la prediccion y el error.** 

Esto deberia verse principalmente como ruido, ya que no deberia haber correlacion entre estos valores.

Finalmente el ultimo grafico que realizaremos es el QQ Plot, que nos indicara la distribucion del error a lo largo de los cuartiles.

**Genere un qq plot con las predicciones realizadas.**
(from statsmodels.graphics.gofplots import qqplot)


En este caso los valores deben concentrarse sobre la linea diagonal.

#### Opcional:

La regularizacion es un metodo mediante el cual se penalisa el valor obsoluto de los coeficientes, de este modo se encuentran modelos mas estables y menos propensos a overfiear los datos.

Implemente una Regresion tipo Lasso (Regularizacion tipo L1) o tipo Ridge (Regularizacion tipo L2), calcula sus metricas y compare con la regresion lineal comun.

## 5. Importancia de las variables, Regresion Lineal

En base a la arquitectura del modelo y utilizando los coeficientes del modelo, determine cuales son las 10 variables de mayor peso en el modelo y explique porque.

## 6. Preparacion de datos para algoritmos basados en arboles

Otra rama muy utilizada de modelos son aquellos basados en arboles de decision, como Bagging, Random Forrest o Boosting. En nuestro caso utilizaremos una implementacion de Boosting por gradiente denominada XGBoost que es reconocida por producir muy buenos resultados.

Dada la naturaleza de los arboles de decision en este caso es necesario normalizar los datos. Para demostrarlo, entrene dos modelos con parametros por default, utilizando el dataset normalizado y sin normalizar.

### Evaluacion del modelo, XGBoost

Seleccione uno de los modelos, calcule las metricas y realize el analisis un analisis de residuos, similar al realizado anteriormente.

## 7. Importancia de las variables

Utilize el metodo "Feature Importance" para determinar las 10 variables mas importante del modelo, verifique los diferentes criterios de calculo, seleccione uno y grafique sus coeficientes.

## 8. (Opcional) Conversion a problema de clasificacion

Podemos cambiar la naturaleza del problema de regresion a clasificacion simplemente cuantizando la salida y asignando una categoria cada intervalo.

Separe la salida en 5 intervalos segun su criterio, asignele categorias de 0 a 5, siendo 0 la peor calidad y 5 la mejor.

Entrene un classificador XGBoost, y calcule las metricas de accuracy, precision, recall, f1 score.

## 9. Conclusiones

A partir de todos los puntos anteriores exponga sus impresiones y conclusiones personales.